# Import

In [1]:
import numpy as np
import pandas as pd
import os

from datetime import datetime

import env
from env import user, password, host

# Make sure we can see the full scale of the data
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Get the data
- pull from SQL
- put in a function that will save data as .csv
- check for null values
- set Order Date as index

In [2]:
query = '''
        SELECT orders.*, products.`Product Name`, categories.Category, categories.`Sub-Category`, regions.`Region Name`
        FROM orders
        LEFT JOIN products ON orders.`Product ID` = products.`Product ID`
        LEFT JOIN categories ON orders.`Category ID` = categories.`Category ID`
        LEFT JOIN regions ON orders.`Region ID` = regions.`Region ID`;
        '''

url = f"mysql+pymysql://{env.user}:{env.password}@{env.host}/superstore_db"

df = pd.read_sql(query, url)
df.head()

,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Segment,Country,City,State,Postal Code,Product ID,Sales,Quantity,Discount,Profit,Category ID,Region ID,Product Name,Category,Sub-Category,Region Name
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-BO-10001798,261.9600,2.0,0.00,41.9136,1,1,Bush Somerset Collection Bookcase,Furniture,Bookcases,South
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-CH-10000454,731.9400,3.0,0.00,219.5820,2,1,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs,South
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,OFF-LA-10000240,14.6200,2.0,0.00,6.8714,3,2,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels,West
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,FUR-TA-10000577,957.5775,5.0,0.45,-383.0310,4,1,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables,South
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,OFF-ST-10000760,22.3680,2.0,0.20,2.5164,5,1,Eldon Fold 'N Roll Cart System,Office Supplies,Storage,South


In [3]:
# Data Types and 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1734 entries, 0 to 1733
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Order ID      1734 non-null   object 
 1   Order Date    1734 non-null   object 
 2   Ship Date     1734 non-null   object 
 3   Ship Mode     1734 non-null   object 
 4   Customer ID   1734 non-null   object 
 5   Segment       1734 non-null   object 
 6   Country       1734 non-null   object 
 7   City          1734 non-null   object 
 8   State         1734 non-null   object 
 9   Postal Code   1734 non-null   float64
 10  Product ID    1734 non-null   object 
 11  Sales         1734 non-null   float64
 12  Quantity      1734 non-null   float64
 13  Discount      1734 non-null   float64
 14  Profit        1734 non-null   float64
 15  Category ID   1734 non-null   int64  
 16  Region ID     1734 non-null   int64  
 17  Product Name  1734 non-null   object 
 18  Category      1734 non-null 

In [4]:
# Shape of data
df.shape

(1734, 21)

In [5]:
# Check for Nulls
df.isnull().sum()

Order ID        0
Order Date      0
Ship Date       0
Ship Mode       0
Customer ID     0
Segment         0
Country         0
City            0
State           0
Postal Code     0
Product ID      0
Sales           0
Quantity        0
Discount        0
Profit          0
Category ID     0
Region ID       0
Product Name    0
Category        0
Sub-Category    0
Region Name     0
dtype: int64

In [6]:
# Quick Glance at Metrics
df.describe()

,Postal Code,Sales,Quantity,Discount,Profit,Category ID,Region ID
count,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000,1734.000000
mean,53501.926182,229.206358,3.794694,0.160110,21.880351,8.106113,2.722607
std,32180.245224,532.862614,2.297407,0.206419,202.891949,3.480593,1.062277
min,1841.000000,1.080000,1.000000,0.000000,-3839.990400,1.000000,1.000000
25%,19143.000000,16.695000,2.000000,0.000000,1.702400,6.000000,2.000000
50%,50315.000000,52.245000,3.000000,0.200000,8.073050,8.000000,3.000000
75%,85705.000000,209.917500,5.000000,0.200000,27.414450,11.000000,4.000000
max,98661.000000,8159.952000,14.000000,0.800000,3177.475000,17.000000,4.000000


In [7]:
df['Order ID'].value_counts()

US-2016-108504    11
CA-2015-131338    10
CA-2017-117457     9
US-2016-110156     8
US-2017-118087     8
CA-2015-119291     7
CA-2014-139892     7
CA-2016-111682     7
US-2015-150630     7
CA-2016-157749     7
CA-2015-124800     7
CA-2016-145583     7
CA-2014-115812     7
CA-2016-128531     6
CA-2017-135279     6
US-2016-115819     6
CA-2016-115756     6
CA-2016-166674     6
CA-2016-126004     6
CA-2014-144666     6
CA-2016-137050     6
US-2014-147627     6
CA-2015-169397     6
CA-2014-126361     6
CA-2017-131954     6
CA-2017-123491     6
CA-2015-161263     6
CA-2017-104220     6
CA-2015-112452     5
CA-2017-117947     5
CA-2014-163552     5
US-2017-155425     5
CA-2014-140858     5
CA-2016-103730     5
CA-2016-152170     5
CA-2014-146969     5
CA-2015-144806     5
CA-2017-135860     5
US-2016-141544     5
CA-2014-149958     5
US-2016-105578     5
CA-2016-142545     5
CA-2014-131926     5
CA-2014-111451     5
CA-2017-117212     5
CA-2016-114727     5
CA-2015-102281     5
US-2017-10093

In [8]:
# Rename Columns then fix the date to index
df = df.rename(columns={
                    'Order ID': 'order_id',
                    'Order Date': 'order_date',
                    'Ship Date': 'ship_date',
                    'Customer ID': 'customer_id',
                    'Postal Code': 'zip_code',
                    'Category ID': 'category_id',
                    'Region ID': 'region_id',
                    'Region Name': 'region_name',
                    'Ship Mode': 'shipping_method'})

df.head()

,order_id,order_date,ship_date,shipping_method,customer_id,Segment,Country,City,State,zip_code,Product ID,Sales,Quantity,Discount,Profit,category_id,region_id,Product Name,Category,Sub-Category,region_name
0,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-BO-10001798,261.9600,2.0,0.00,41.9136,1,1,Bush Somerset Collection Bookcase,Furniture,Bookcases,South
1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Consumer,United States,Henderson,Kentucky,42420.0,FUR-CH-10000454,731.9400,3.0,0.00,219.5820,2,1,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",Furniture,Chairs,South
2,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Corporate,United States,Los Angeles,California,90036.0,OFF-LA-10000240,14.6200,2.0,0.00,6.8714,3,2,Self-Adhesive Address Labels for Typewriters b...,Office Supplies,Labels,West
3,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,FUR-TA-10000577,957.5775,5.0,0.45,-383.0310,4,1,Bretford CR4500 Series Slim Rectangular Table,Furniture,Tables,South
4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Consumer,United States,Fort Lauderdale,Florida,33311.0,OFF-ST-10000760,22.3680,2.0,0.20,2.5164,5,1,Eldon Fold 'N Roll Cart System,Office Supplies,Storage,South


In [13]:
# Set order_date as the index

df['order_date'] = pd.to_datetime(df['order_date'])

df = df.set_index('order_date').sort_index()

In [14]:
df.head()

,order_id,ship_date,shipping_method,customer_id,Segment,Country,City,State,zip_code,Product ID,Sales,Quantity,Discount,Profit,category_id,region_id,Product Name,Category,Sub-Category,region_name
order_date,,,,,,,,,,,,,,,,,,,,
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,PO-19195,Home Office,United States,Naperville,Illinois,60540.0,OFF-LA-10003223,11.784,3.0,0.2,4.2717,3,3,Avery 508,Office Supplies,Labels,Central
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,PO-19195,Home Office,United States,Naperville,Illinois,60540.0,OFF-BI-10004094,3.540,2.0,0.8,-5.4870,9,3,GBC Standard Plastic Binding Systems Combs,Office Supplies,Binders,Central
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,PO-19195,Home Office,United States,Naperville,Illinois,60540.0,OFF-ST-10002743,272.736,3.0,0.2,-64.7748,5,3,SAFCO Boltless Steel Shelving,Office Supplies,Storage,Central
2014-01-09,CA-2014-135405,2014-01-13,Standard Class,MS-17830,Consumer,United States,Laredo,Texas,78041.0,TEC-AC-10001266,31.200,3.0,0.2,9.7500,12,3,Memorex Micro Travel Drive 8 GB,Technology,Accessories,Central
2014-01-09,CA-2014-135405,2014-01-13,Standard Class,MS-17830,Consumer,United States,Laredo,Texas,78041.0,OFF-AR-10004078,9.344,2.0,0.2,1.1680,7,3,Newell 312,Office Supplies,Art,Central


## Takeaways thus far:
- No missing values initially present
- 1,734 rows
- 841 seperate orders


In [15]:
def wrangle_superstore():
    '''
    '''
    
    filename = 'superstore.csv'
    
    if os.path.exists(filename):
        print('Reading cleaned data from csv file...')
        return pd.read_csv(filename)
    
    # Pull from SQL
    query = '''
        SELECT orders.*, products.`Product Name`, categories.Category, categories.`Sub-Category`, regions.`Region Name`
        FROM orders
        LEFT JOIN products ON orders.`Product ID` = products.`Product ID`
        LEFT JOIN categories ON orders.`Category ID` = categories.`Category ID`
        LEFT JOIN regions ON orders.`Region ID` = regions.`Region ID`;
        '''

    url = f"mysql+pymysql://{env.user}:{env.password}@{env.host}/superstore_db"

    df = pd.read_sql(query, url)
    
    # Rename columns
    df = df.rename(columns={
                    'Order ID': 'order_id',
                    'Order Date': 'order_date',
                    'Ship Date': 'ship_date',
                    'Customer ID': 'customer_id',
                    'Postal Code': 'zip_code',
                    'Category ID': 'category_id',
                    'Region ID': 'region_id',
                    'Region Name': 'region_name',
                    'Ship Mode': 'shipping_method',
                    'Sub-Category': 'sub_category'})
    
    df.columns= df.columns.str.lower()
    
    # Set order_date to index
    df['order_date'] = pd.to_datetime(df['order_date'])

    df = df.set_index('order_date').sort_index()
    
    # Download cleaned data to a .csv
    df.to_csv(filename, index=False)
    
    print('Downloading data from SQL...')
    print('Saving to .csv')
    return df

In [16]:
df = wrangle_superstore()
df.head()

Saving to .csv


,order_id,ship_date,shipping_method,customer_id,segment,country,city,state,zip_code,product id,sales,quantity,discount,profit,category_id,region_id,product name,category,sub-category,region_name
order_date,,,,,,,,,,,,,,,,,,,,
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,PO-19195,Home Office,United States,Naperville,Illinois,60540.0,OFF-LA-10003223,11.784,3.0,0.2,4.2717,3,3,Avery 508,Office Supplies,Labels,Central
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,PO-19195,Home Office,United States,Naperville,Illinois,60540.0,OFF-BI-10004094,3.540,2.0,0.8,-5.4870,9,3,GBC Standard Plastic Binding Systems Combs,Office Supplies,Binders,Central
2014-01-04,CA-2014-112326,2014-01-08,Standard Class,PO-19195,Home Office,United States,Naperville,Illinois,60540.0,OFF-ST-10002743,272.736,3.0,0.2,-64.7748,5,3,SAFCO Boltless Steel Shelving,Office Supplies,Storage,Central
2014-01-09,CA-2014-135405,2014-01-13,Standard Class,MS-17830,Consumer,United States,Laredo,Texas,78041.0,TEC-AC-10001266,31.200,3.0,0.2,9.7500,12,3,Memorex Micro Travel Drive 8 GB,Technology,Accessories,Central
2014-01-09,CA-2014-135405,2014-01-13,Standard Class,MS-17830,Consumer,United States,Laredo,Texas,78041.0,OFF-AR-10004078,9.344,2.0,0.2,1.1680,7,3,Newell 312,Office Supplies,Art,Central


## Value counts of select columns

In [17]:
df.shipping_method.value_counts()

Standard Class    980
Second Class      354
First Class       326
Same Day           74
Name: shipping_method, dtype: int64

In [18]:
df.segment.value_counts()

Consumer       954
Corporate      478
Home Office    302
Name: segment, dtype: int64

In [24]:
df.city.value_counts()

New York City       192
Los Angeles         124
Philadelphia        123
San Francisco        96
Seattle              62
Houston              58
Chicago              51
Columbus             42
Dallas               26
San Diego            24
Detroit              23
Jacksonville         19
Springfield          18
Tampa                15
Denver               15
Tucson               15
Aurora               15
San Antonio          14
Jackson              14
Lakeville            13
Smyrna               12
Phoenix              12
Mesa                 12
Pasadena             12
Charlotte            12
Chester              11
Richmond             11
Fort Worth           11
Rochester            10
Cincinnati           10
Wilmington            9
Florence              9
Toledo                9
Cleveland             9
Auburn                9
Louisville            9
Des Moines            9
Troy                  8
Henderson             8
Newark                8
Arlington             8
Long Beach      

In [25]:
df.discount.value_counts()

0.00    810
0.20    656
0.70     82
0.80     46
0.40     41
0.30     31
0.60     19
0.50     18
0.10     14
0.15      8
0.45      5
0.32      4
Name: discount, dtype: int64

In [26]:
df.category.value_counts()

Office Supplies    1055
Furniture           360
Technology          319
Name: category, dtype: int64

In [ ]:
df.sub-categor